In [5]:
:set -XOverloadedStrings

import qualified Data.ByteString as B
import qualified Data.ByteString.UTF8

import Control.Monad.Trans.Reader
import Control.Monad.IO.Class (liftIO)
import Control.Monad (mapM_, unless)
import System.Directory
import System.FilePath

import Duffer.Loose
import Duffer.Loose.Objects

duffer = flip runReaderT "output/.git"
git    = flip runReaderT ".git"

In [9]:
Just o <- git$ readObject "b1bbf2b4b38a53c8b986d30b0e8f60ff8c01505f"
authorTime o
initRepo = do
    path <- ask
    objectsPresent <- liftIO $ doesDirectoryExist $ path </> "objects"
    liftIO $ unless objectsPresent $ do
        mapM_ (createDirectoryIfMissing True)
            [ path </> "branches"
            , path </> "hooks"
            , path </> "info"
            , path </> "objects" </> "info"
            , path </> "objects" </> "pack"
            , path </> "refs"    </> "heads"
            , path </> "refs"    </> "tags"
            ]
        mapM_ (uncurry writeFile)
            [ (path </> "HEAD",   "ref: refs/heads/master\n")
            , (path </> "config", "")
            , (path </> "description", "")
            ]

destroyRepo = do
    path <- ask
    liftIO $ removeDirectoryRecursive path


_ <- duffer initRepo

No instance for (Show PersonTime) arising from a use of ‘print’
In a stmt of an interactive GHCi command: print it

In [23]:
import Data.Byteable
import qualified Data.Set as S
me = PersonTime "Vaibhav Sagar" "vaibhavsagar@gmail" "0000000000" "-0400"
toBytes me

master parent = do
    message           <- liftIO $ B.readFile "content/step0/commit.txt"
    resolveRefContent <- liftIO $ B.readFile "content/step0/tree/tools/resolve_ref.py"
    resolveRefHash    <- writeObject $ Blob resolveRefContent
    toolsSubtreeHash  <- writeObject $ Tree $ S.fromList [TreeEntry 0o100644 "resolve_ref.py" resolveRefHash]
    rootTreeHash      <- writeObject $ Tree $ S.fromList [TreeEntry 0o040000 "tools" toolsSubtreeHash]
    let commitObject  =  Commit rootTreeHash [parent] me me message
    writeObject commitObject
    return commitObject

stepN step parents = do
    message           <- liftIO $ B.readFile $ "content/step" ++ n ++ "/commit.txt"
    resolveRefContent <- liftIO $ B.readFile $ "content/step" ++ n ++ "/tree/tools/resolve_ref.py"
    showObjectContent <- liftIO $ B.readFile $ "content/step" ++ n ++ "/tree/tools/show_object.py"
    resolveRefHash    <- writeObject $ Blob resolveRefContent
    showObjectHash    <- writeObject $ Blob showObjectContent
    toolsSubtreeHash  <- writeObject $ Tree $ S.fromList 
        [ TreeEntry 0o100644 "resolve_ref.py" resolveRefHash
        , TreeEntry 0o100644 "show_object.py" showObjectHash
        ]
    rootTreeHash      <- writeObject $ Tree $ S.fromList [TreeEntry 0o040000 "tools" toolsSubtreeHash]
    let commitObject  = Commit rootTreeHash parents me me message
    commitHash        <- writeObject $ commitObject
    updateRef ("refs/tags/step" ++ n) commitObject
    return commitHash
    where n = show step

step5 parent = do
    message           <- liftIO $ B.readFile $ "content/step" ++ n ++ "/commit.txt"
    resolveRefContent <- liftIO $ B.readFile $ "content/step" ++ n ++ "/tree/tools/resolve_ref.py"
    showObjectContent <- liftIO $ B.readFile $ "content/step" ++ n ++ "/tree/tools/show_object.py"
    showObjectContent <- liftIO $ B.readFile $ "content/step" ++ n ++ "/tree/tools/show_object.py"
    unpackContent     <- liftIO $ B.readFile $ "content/step" ++ n ++ "/tree/tools/unpack.sh"
    resolveRefHash    <- writeObject $ Blob resolveRefContent
    showObjectHash    <- writeObject $ Blob showObjectContent
    unpackHash        <- writeObject $ Blob unpackContent
    toolsSubtreeHash  <- writeObject $ Tree $ S.fromList 
        [ TreeEntry 0o100644 "resolve_ref.py" resolveRefHash
        , TreeEntry 0o100644 "show_object.py" showObjectHash
        , TreeEntry 0o100644 "unpack.sh"      unpackHash
        ]
    rootTreeHash      <- writeObject $ Tree $ S.fromList [TreeEntry 0o040000 "tools" toolsSubtreeHash]
    commitHash        <- writeObject $ Commit rootTreeHash [parent] me me message
    return commitHash
    where n = show 5

makeRepo = do
    latest <- stepN 1 [] >>= stepN 2 . return >>= stepN 3 . return >>= stepN 4 .return >>= stepN 5 . return >>= master
    updateRef "refs/heads/master" latest
    return latest

:t makeRepo
_ <- duffer makeRepo

"Vaibhav Sagar <vaibhavsagar@gmail> 0000000000 -0400"

makeRepo :: ReaderT Repo IO GitObject